In [50]:
import requests
headers = {
    'accept': 'application/json, text/plain, */*',
    'accept-language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
    'origin': 'https://www.nasdaq.com',
    'referer': 'https://www.nasdaq.com/',
    'user-agent': 'Mozilla/5.0 (Macintosh; ARM64 Mac OS X 13_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36',
}
res = requests.get('https://api.nasdaq.com/api/screener/stocks?tableonly=true&limit=25&offset=0&download=true', headers=headers)
res

<Response [200]>

In [51]:
assert 'data' in res.json()
data = res.json()['data']
data

{'asOf': None,
 'headers': {'symbol': 'Symbol',
  'name': 'Name',
  'lastsale': 'Last Sale',
  'netchange': 'Net Change',
  'pctchange': '% Change',
  'marketCap': 'Market Cap',
  'country': 'Country',
  'ipoyear': 'IPO Year',
  'volume': 'Volume',
  'sector': 'Sector',
  'industry': 'Industry',
  'url': 'Url'},
 'rows': [{'symbol': 'A',
   'name': 'Agilent Technologies Inc. Common Stock',
   'lastsale': '$137.47',
   'netchange': '0.47',
   'pctchange': '0.343%',
   'volume': '1369873',
   'marketCap': '39260786166.00',
   'country': 'United States',
   'ipoyear': '1999',
   'industry': 'Biotechnology: Laboratory Analytical Instruments',
   'sector': 'Industrials',
   'url': '/market-activity/stocks/a'},
  {'symbol': 'AA',
   'name': 'Alcoa Corporation Common Stock ',
   'lastsale': '$35.91',
   'netchange': '-0.09',
   'pctchange': '-0.25%',
   'volume': '2942573',
   'marketCap': '9277521873.00',
   'country': 'United States',
   'ipoyear': '2016',
   'industry': 'Aluminum',
   'sec

In [52]:
import pandas as pd
df = pd.DataFrame(data['rows'])
df

,symbol,name,lastsale,netchange,pctchange,volume,marketCap,country,ipoyear,industry,sector,url
0,A,Agilent Technologies Inc. Common Stock,$137.47,0.47,0.343%,1369873,39260786166.00,United States,1999,Biotechnology: Laboratory Analytical Instruments,Industrials,/market-activity/stocks/a
1,AA,Alcoa Corporation Common Stock,$35.91,-0.09,-0.25%,2942573,9277521873.00,United States,2016,Aluminum,Industrials,/market-activity/stocks/aa
2,AACG,ATA Creativity Global American Depositary Shares,$0.9462,-0.0329,-3.36%,20957,30278523.00,China,2008,Other Consumer Services,Real Estate,/market-activity/stocks/aacg
3,AACT,Ares Acquisition Corporation II Class A Ordina...,$10.98,0.00,0.00%,14192,0.00,,2023,Blank Checks,Finance,/market-activity/stocks/aact
4,AADI,Aadi Bioscience Inc. Common Stock,$3.60,0.00,0.00%,566757,88730611.00,United States,,Biotechnology: Pharmaceutical Preparations,Health Care,/market-activity/stocks/aadi
...,...,...,...,...,...,...,...,...,...,...,...,...
6977,ZVSA,ZyVersa Therapeutics Inc. Common Stock,$1.47,0.29,24.576%,38269055,3445961.00,United States,2022,Biotechnology: Pharmaceutical Preparations,Health Care,/market-activity/stocks/zvsa
6978,ZWS,Zurn Elkay Water Solutions Corporation Common ...,$36.20,-0.76,-2.056%,682876,6143642890.00,United States,2012,Industrial Machinery/Components,Industrials,/market-activity/stocks/zws
6979,ZYBT,Zhengye Biotechnology Holding Limited Ordinary...,$4.6162,0.1162,2.582%,502143,217729425.00,China,2025,Biotechnology: Pharmaceutical Preparations,Health Care,/market-activity/stocks/zybt
6980,ZYME,Zymeworks Inc. Common Stock,$13.22,-0.91,-6.44%,798973,910560894.00,United States,,Biotechnology: Pharmaceutical Preparations,Health Care,/market-activity/stocks/zyme


In [53]:
df[df['symbol'] == 'AAPL']

,symbol,name,lastsale,netchange,pctchange,volume,marketCap,country,ipoyear,industry,sector,url
12,AAPL,Apple Inc. Common Stock,$236.85,-5.85,-2.41%,61678663,3580182677550.00,United States,1980,Computer Manufacturing,Technology,/market-activity/stocks/aapl
